#Crime Category Prediction Using Machine Learning

- Spark ML's Machine Learning Models were used to predict the crime "Category" from the input feature vector consisted of ["DayOfWeek", "Month", "Date", "Time", "PdDistrict", "X", "Y"]
- Random Forest Classifier Model and the Multi-layer Perception Classifier Model were used

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark import SparkContext

In [3]:
sqlContext = SQLContext(sc)

In [4]:
dataPath = '/FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv'
crimeDataSchema = StructType([StructField("IncidntNum", LongType(), True),
                              StructField("Category", StringType(), True),
                              StructField("Descript", StringType(), True),
                              StructField("DayOfWeek", StringType(), True),
                              StructField("Date", StringType(), True),
                              StructField("Time", StringType(), True),
                              StructField("PdDistrict", StringType(), True),
                              StructField("Resolution", StringType(), True),
                              StructField("Address", StringType(), True),
                              StructField("X", DoubleType(), True),
                              StructField("Y", DoubleType(), True),
                              StructField("Location", StringType(), True),
                              StructField("PdId", LongType(), True)])

crimeDF = (sqlContext.read
           .format('csv')
           .option('delimiter', ',')
           .option('header', 'true')
           .load(dataPath, schema=crimeDataSchema))

In [5]:
crimeDF = crimeDF.filter("PdDistrict is not NULL")

In [6]:
cats = crimeDF.select("Category").distinct().collect()

In [7]:
Category_idx = {}

for i in range(39):
    Category_idx.update({cats[i][0]:i})

In [8]:
day_idx = {"Monday":0,
           "Tuesday":1,
           "Wednesday":2,
           "Thursday":3,
           "Friday":4,
           "Saturday":5,
           "Sunday":6
          }

district_idx = {"MISSION":0,
                "BAYVIEW":1,
                "CENTRAL":2,
                "TARAVAL":3,
                "TENDERLOIN":4,
                "INGLESIDE":5,
                "PARK":6,
                "SOUTHERN":7,
                "RICHMOND":8,
                "NORTHERN":9
               }

For the dataset,
- "Date" column is parsed and decomposed into Month and Date
- "Time" column is parsed to extract out the hour from the time
- "PdDistrict" is encoded with district_idx
- "DayOfWeek" is encoded following day_idx
- "Category" is encoded with Category_idx

In [10]:
import datetime

# def parseDate(dateStr):
#     tokens = dateStr.split("/")
    
#     month = int(tokens[0])
#     date = int(tokens[1])
#     year = int(tokens[2])
#     return datetime.date(year, month, date)
  
def parseMonth(dateStr):
    tokens = dateStr.split("/")    
    return int(tokens[0])

def parseDate(dateStr):
    tokens = dateStr.split("/")    
    return int(tokens[1])

def parseTime(timeStr):
    tokens = timeStr.split(":")
    hour = int(tokens[0])
    minute = int(tokens[1])
    return hour

def parseDay(dayStr):
    return day_idx[dayStr]

def parseDist(distStr):
    return district_idx[distStr]

def parseCategory(catStr):
    return Category_idx[catStr]

from pyspark.sql.functions import udf

crimeDF = (crimeDF.withColumn("Date_tmp", udf(parseDate, IntegerType())(crimeDF.Date))
           .withColumn("Month_tmp", udf(parseMonth, IntegerType())(crimeDF.Date))
           .withColumn("Time_tmp", udf(parseTime, IntegerType())(crimeDF.Time))
           .withColumn("Day_tmp", udf(parseDay, IntegerType())(crimeDF.DayOfWeek))
           .withColumn("District_tmp", udf(parseDist, IntegerType())(crimeDF.PdDistrict))
           .withColumn("Cats_tmp", udf(parseCategory, IntegerType())(crimeDF.Category))
           .drop("Time")
           .drop("Date")
           .drop("DayOfWeek")
           .drop("PdDistrict")
           .drop("Category")
           .withColumnRenamed("Date_tmp", "Date")
           .withColumnRenamed("Month_tmp", "Month")
           .withColumnRenamed("Time_tmp", "Time")
           .withColumnRenamed("Day_tmp", "DayOfWeek")
           .withColumnRenamed("District_tmp", "PdDistrict")
           .withColumnRenamed("Cats_tmp", "Category")
          ).cache()

In [11]:
crimeDF.show()

+----------+--------------------+--------------+--------------------+-----------------+----------------+--------------------+--------------+----+-----+----+---------+----------+--------+
IncidntNum| Descript| Resolution| Address| X| Y| Location| PdId|Date|Month|Time|DayOfWeek|PdDistrict|Category|
+----------+--------------------+--------------+--------------------+-----------------+----------------+--------------------+--------------+----+-----+----+---------+----------+--------+
 150060275| LOST PROPERTY| NONE|18TH ST / VALENCI...| -122.42158168137|37.7617007179518|(37.7617007179518...|15006027571000| 19| 1| 14| 0| 0| 32|
 150098210|ROBBERY, BODILY F...| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821003074| 1| 2| 15| 6| 4| 20|
 150098210|AGGRAVATED ASSAUL...| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821004014| 1| 2| 15| 6| 4| 13|
 150098210| DOMESTIC VIOLENCE| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821015200| 1| 2| 15| 6| 4| 3|
 150098226|MALICIOUS MISCHIE...| NONE|LOMBARD ST / LAGU...|-122.431118543788|37.8004687042875|(37.8004687042875...|15009822628160| 27| 1| 19| 1| 9| 28|
 150098232|AIDED CASE -PROPE...| NONE|400 Block of LOCU...|-122.451781767894|37.7870853907529|(37.7870853907529...|15009823251041| 1| 2| 16| 6| 8| 32|
 150098248| DOMESTIC VIOLENCE| NONE|700 Block of KIRK...|-122.374019331833| 37.729203356539|(37.729203356539,...|15009824815200| 31| 1| 21| 5| 1| 3|
 150098248|MALICIOUS MISCHIE...| NONE|700 Block of KIRK...|-122.374019331833| 37.729203356539|(37.729203356539,...|15009824828150| 31| 1| 21| 5| 1| 28|
 150098254|BURGLARY OF STORE...| NONE|200 Block of STOC...| -122.40656817787|37.7878092959561|(37.7878092959561...|15009825405053| 31| 1| 16| 5| 2| 37|
 150098260|PETTY THEFT SHOPL...| NONE|800 Block of GEAR...|-122.417295322526|37.7862578545865|(37.7862578545865...|15009826006362| 31| 1| 17| 5| 2| 35|
 150098345|PETTY THEFT SHOPL...|ARREST, BOOKED|1700 Block of HAR...|-122.413354187018|37.7690748003847|(37.7690748003847...|15009834506362| 1| 2| 14| 6| 0| 35|
 150098345|POSSESSION OF MET...|ARREST, BOOKED|1700 Block of HAR...|-122.413354187018|37.7690748003847|(37.7690748003847...|15009834516650| 1| 2| 14| 6| 0| 29|
 150098345|POSSESSION OF NAR...|ARREST, BOOKED|1700 Block of HAR...|-122.413354187018|37.7690748003847|(37.7690748003847...|15009834516710| 1| 2| 14| 6| 0| 29|
 150098345| WARRANT ARREST|ARREST, BOOKED|1700 Block of HAR...|-122.413354187018|37.7690748003847|(37.7690748003847...|15009834563010| 1| 2| 14| 6| 0| 8|
 150098367|ROBBERY, ARMED WI...| NONE|2200 Block of MAR...|-122.432198022433|37.7651107322703|(37.7651107322703...|15009836703072| 1| 2| 16| 6| 0| 20|
 150098373| STOLEN AUTOMOBILE| NONE|FRANKLIN ST / TUR...| -122.42227702764|37.7817528767881|(37.7817528767881...|15009837307021| 1| 2| 12| 6| 9| 33|
 150098373| FOUND PROPERTY| NONE|FRANKLIN ST / TUR...| -122.42227702764|37.7817528767881|(37.7817528767881...|15009837372000| 1| 2| 12| 6| 9| 32|
 150098389|ENROUTE TO OUTSID...|ARREST, BOOKED|3800 Block of 3RD ST|-122.387939452192|37.7422600519555|(37.7422600519555...|15009838962050| 1| 2| 16| 6| 1| 8|
 150098395|PETTY THEFT FROM ...| NONE|MARTIN LUTHER KIN...|-122.458638758608|37.7671999403456|(37.7671999403456...|15009839506243| 1| 2| 14| 6| 6| 35|
 150098408|DRIVERS LICENSE, ...|ARREST, BOOKED|NEWCOMB AV / MEND...|-122.389774683064| 37.735226586208|(37.735226586208,...|15009840865016| 1| 2| 17| 6| 1| 6|
+----------+--------------------+--------------+--------------------+-----------------+----------------+--------------------+--------------+----+-----+----+---------+----------+--------+
only showing top 20 rows

#### Input Feature

In [13]:
from pyspark.ml.feature import VectorAssembler

vectorizer = VectorAssembler()
vectorizer.setInputCols(["DayOfWeek", "Month", "Date", "Time", "PdDistrict", "X", "Y"])
vectorizer.setOutputCol("features")


Out[ 12 ]: VectorAssembler_4c07acd1315ccf218e28

#### Training, Test Dataset Splitting

In [15]:
(split20DF, split80DF) = crimeDF.randomSplit([20.0, 80.0])

testSetDF = split20DF.cache()
trainingSetDF = split80DF.cache()

####Classification Evaluator

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

classEval = MulticlassClassificationEvaluator(predictionCol="Prediction_Category", labelCol="Category", metricName="accuracy")

##Random Forest Classifier
- 3-fold crossvalidation 
- Grid-search on "maxBins", "maxDepth", "numTrees" paramters

In [19]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

rf = RandomForestClassifier()
rf.setParams(featuresCol="features", labelCol="Category", predictionCol="Prediction_Category")

rfPipeline = Pipeline()
rfPipeline.setStages([vectorizer, rf])

Out[ 21 ]: Pipeline_4f28a0e4a68d6028d979

In [20]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

crossval = CrossValidator(estimator=rfPipeline, evaluator=classEval, numFolds=3)

paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxBins, [50,100])
            .addGrid(rf.maxDepth, [5, 8])
            .addGrid(rf.numTrees, [20, 25])
            .build())

crossval.setEstimatorParamMaps(paramGrid)
rfModel = crossval.fit(trainingSetDF).bestModel

In [21]:
resultsDF = rfModel.transform(testSetDF)

In [22]:
evalRF = classEval.evaluate(resultsDF)

In [23]:
print("RF Accuracy: {0:.2f}".format(evalRF))

RF Accuracy: 0.26

##Multilayer Perceptron Classifier
- consists of 4 layers of size [7, 50, 39]
  - input feature consists of 7 elements
  - output is 1 of 39 different categories

In [25]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline

layers = [7, 50, 39]

mlp = MultilayerPerceptronClassifier().setLayers(layers).setLabelCol("Category").setFeaturesCol("features").setPredictionCol("Prediction_Category")

mlpPipeline = Pipeline()
mlpPipeline.setStages([vectorizer, mlp])

mlpModel = mlpPipeline.fit(trainingSetDF)

In [26]:
resultsMLP = mlpModel.transform(testSetDF)

In [27]:
evalMLP = classEval.evaluate(resultsMLP)

In [28]:
print("MLP Accuracy: {0:.2f}".format(evalMLP))

MLP Accuracy: 0.22